# 10K Analysis
In this demo, we explore answering complex queries by decomposing them into simpler sub-queries.

In [11]:
import nest_asyncio
nest_asyncio.apply()

In [36]:
from llama_index import SimpleDirectoryReader, LLMPredictor, ServiceContext, GPTVectorStoreIndex
from llama_index.response.pprint_utils import pprint_response
from langchain import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

## Configure LLM service

In [2]:
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=-1, streaming=True))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

## Load data 

In [3]:
lyft_docs = SimpleDirectoryReader(input_files=["../data/10k/lyft_2021.pdf"]).load_data()
uber_docs = SimpleDirectoryReader(input_files=["../data/10k/uber_2021.pdf"]).load_data()

## Build indices

In [4]:
lyft_index = GPTVectorStoreIndex.from_documents(lyft_docs)

In [5]:
uber_index = GPTVectorStoreIndex.from_documents(uber_docs)

## Build query engines

In [6]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)

In [7]:
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [23]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine, 
        metadata=ToolMetadata(name='lyft_10k', description='Provides information about Lyft financials for year 2021')
    ),
    QueryEngineTool(
        query_engine=uber_engine, 
        metadata=ToolMetadata(name='uber_10k', description='Provides information about Uber financials for year 2021')
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

## Run queries

In [24]:
response = s_engine.query('Compare and contrast the customer segments and geographies that grew the fastest')

Generated 4 sub questions.
[uber_10k] Q: What customer segments grew the fastest for Uber
[uber_10k] A: 
Uber's Delivery offering grew the fastest, as it allowed consumers to search for and discover local restaurants, order a meal, and either pick-up at the restaurant or have the meal delivered. Additionally, Delivery also includes offerings for grocery, alcohol and convenience store delivery as well as select other goods.
[uber_10k] Q: What geographies grew the fastest for Uber
[uber_10k] A: 
Uber experienced the fastest growth in Mobility Gross Bookings in five metropolitan areas—Chicago, Miami, and New York City in the United States, Sao Paulo in Brazil, and London in the United Kingdom. Additionally, Uber experienced strong competition in large metropolitan areas, which led to the offering of significant Driver incentives and consumer discounts and promotions in these large metropolitan areas.
[lyft_10k] Q: What customer segments grew the fastest for Lyft
[lyft_10k] A: 
The custome

In [25]:
print(response)


Uber experienced the fastest growth in Mobility Gross Bookings in five metropolitan areas—Chicago, Miami, New York City in the United States, Sao Paulo in Brazil, and London in the United Kingdom. The customer segments that grew the fastest for Lyft were riders and drivers. Lyft has grown rapidly in the United States and Canada, and has also expanded its offerings to select cities. 

Both Uber and Lyft have experienced the fastest growth in metropolitan areas, however, Uber's growth has been concentrated in five cities while Lyft's growth has been more widespread across the United States and Canada. Additionally, Uber's customer segments that grew the fastest were related to delivery services, while Lyft's customer segments that grew the fastest were riders and drivers.


In [28]:
response = s_engine.query('Compare revenue growth of Uber and Lyft from 2020 to 2021')

Generated 2 sub questions.
[uber_10k] Q: What is the revenue growth of Uber from 2020 to 2021
[uber_10k] A: 
The revenue growth of Uber from 2020 to 2021 was 57%, or 54% on a constant currency basis.
[lyft_10k] Q: What is the revenue growth of Lyft from 2020 to 2021
[lyft_10k] A: 
The revenue growth of Lyft from 2020 to 2021 was 36.7%, increasing from $2,364,681 thousand to $3,208,323 thousand.


In [29]:
print(response)


The revenue growth of Uber from 2020 to 2021 was higher than that of Lyft, at 57% or 54% on a constant currency basis, compared to Lyft's 36.7%.
